### Task (12.5pts)

Task description

You will write the K-Means and Bradley-Fayyad-Reina (BFR) algorithms from scratch. You should implement K-Means as the in-memory clustering algorithm that you will use in BFR. You will iteratively
load the data points from a file and process these data points with the BFR algorithm. See below pseudocode for your reference.

    for file in input_path:
        data_points = load(file)
        if first round:
            run K-means for initialization
        else:
            run BFR(data_points)

In BFR, there are three sets of points that you need to keep track of: Discard set (DS), Compression set (CS), Retained set (RS). For each cluster in the DS and CS, the cluster is summarized by:

- N: The number of points
- SUM: the sum of the coordinates of the points
- SUMSQ: the sum of squares of coordinates

In [1]:
# import sys
import time
import json
import math
import random
import copy
from collections import defaultdict
from pyspark import SparkContext

In [8]:
# params = sys.argv
# input_path, n_cluster = params[1], int(params[2])
# out_file1, out_file2 = params[3], params[4]
input_path = ["./test1/data0.txt", 
              "./test1/data1.txt", 
              "./test1/data2.txt", 
              "./test1/data3.txt", 
              "./test1/data4.txt"]
n_cluster = 3

In [3]:
sc = SparkContext.getOrCreate()

In [ ]:
# for i,f in enumerate(sorted(input_path)):
#     data = sc.textFile(f).map(lambda r: r.split(",")) \
#                         .map(lambda kvs: (int(kvs[0]), list(map(kvs[1:]))))
    
#     if i==0:
        

In [4]:
data_points = sc.textFile(input_path[0]).map(lambda r: r.split(",")) \
                                    .map(lambda kvs: (int(kvs[0]), tuple(map(eval, kvs[1:]))))#.collectAsMap()

In [6]:
# data_points[0]
# points = [list(p) for i,p in data_points.items()]
# points[:2]

[[-51.71899392687148,
  -16.014919500749066,
  -32.74523700769919,
  -18.521163057290334,
  24.521957915324595,
  7.929585733451519,
  41.607876608466114,
  4.632327234431364,
  -17.122746655394742,
  -31.53526513234625],
 [-94.94707151240083,
  378.96690376908157,
  -277.579652049817,
  11.187474344193511,
  -99.69220665825787,
  437.0080120495583,
  390.92621884458896,
  227.73387254121732,
  17.823374803974243,
  208.9175743945834]]

In [5]:
n_data = data_points.count()
n_sample = 10000 if n_data > 10000 else int(n_data *.1)
data_sample = data_points.filter(lambda kv: kv[0]<n_sample).collectAsMap()
points = [list(p) for i,p in data_sample.items()]

In [6]:
def calc_distances(A, B, std=None, distance_type="euclidean"):
    if distance_type=="euclidean":
        return float(math.sqrt(sum([(a - b)**2 for a,b in zip(A, B)])))
    elif distance_type=="mahalanobis":
        return float(math.sqrt(sum([((a - b)/sd)**2 for a,b,sd in zip(A, B, std)])))

In [10]:
class KMeans():
    def __init__(self, n_clusters, max_iter=300, tol=1e-4, seed=41):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.seed = seed
        self.cluster_stat = defaultdict(dict)

    def _init_centroids(self, X, init_type='random'):
        random.seed(self.seed)
        n_clusters = self.n_clusters
        n_samples, n_features = len(X), len(X[0])
        centroids_clusters = {}

        if init_type=='random':
            centroids = random.sample(range(n_samples), n_clusters)
            for i,c in enumerate(centroids):
                centroids_clusters[tuple(X[c])] = i
#         elif init_type=='dispersed': ######################

        return centroids_clusters

    def _find_close_cluster(self, x, centroids):
        dist_min = float('inf')
        for c in centroids:
            dist_cur = calc_distances(x, c)
            if dist_cur < dist_min:
                dist_min = dist_cur
                centroid_close = c
                
        return centroid_close
    
    def _update_centroids(self, clusters_points):
        cluster_stat_prev = copy.deepcopy(self.cluster_stat)
        
        for clst,ps in clusters_points.items():
            stat = {'N': len(ps),
                   'SUM': [sum(i) for i in list(map(list, zip(*ps)))],
                   'SUMSQ': [sum([v**2 for v in i]) for i in list(map(list, zip(*ps)))]}
            stat['centroid'] = tuple([s/stat['N'] for s in stat['SUM']])
            self.cluster_stat[clst] = stat
            
        return cluster_stat_prev, self.cluster_stat
    
    
    def fit(self, X):
        n_clusters = self.n_clusters
        centroids_clusters = self._init_centroids(X)
        n_iter = 0
        points_clusters = {}

        while n_iter <= self.max_iter:
            clusters_points = defaultdict(list)
            n_iter += 1
            for x in X:
                centroid_close = self._find_close_cluster(x, centroids_clusters.keys())
                cluster_close = centroids_clusters[centroid_close]
                points_clusters[tuple(x)] = cluster_close
                clusters_points[cluster_close].append(x)

            cluster_stat_prev, self.cluster_stat = self._update_centroids(clusters_points)
            
            if n_iter>1:
                movement = [calc_distances(cluster_stat_prev[clst]['centroid'], stat['centroid']) \
                                     for clst,stat in self.cluster_stat.items()]
                if sum(movement)<=self.tol:
                    break
        return self.cluster_stat, points_clusters

In [11]:
kmeans = KMeans(n_clusters)
cluster_stat, points_clusters = kmeans.fit(points)
points_clusters

{(-51.71899392687148,
  -16.014919500749066,
  -32.74523700769919,
  -18.521163057290334,
  24.521957915324595,
  7.929585733451519,
  41.607876608466114,
  4.632327234431364,
  -17.122746655394742,
  -31.53526513234625): 1,
 (-94.94707151240083,
  378.96690376908157,
  -277.579652049817,
  11.187474344193511,
  -99.69220665825787,
  437.0080120495583,
  390.92621884458896,
  227.73387254121732,
  17.823374803974243,
  208.9175743945834): 1,
 (-51.66224282196675,
  -16.19516936340337,
  -33.586236537259566,
  -18.979721131461726,
  26.932542653892973,
  7.959141763861954,
  40.28729912953935,
  4.4623191502937996,
  -16.88867354864481,
  -30.753003762523015): 1,
 (242.11376483349872,
  92.51815402450225,
  367.85860708533335,
  -278.2591099007162,
  414.74597017072284,
  -46.2801536514048,
  -56.532193126064804,
  -233.38165487461836,
  -246.31972404619088,
  -136.9751540047138): 1,
 (241.53581703205816,
  98.10070576626687,
  368.3622397166624,
  -272.99055418834524,
  413.68707576691

In [12]:
cluster_stat

defaultdict(dict,
            {1: {'N': 8645,
              'SUM': [-80760.08857665949,
               -105335.69627178095,
               -213953.53085151143,
               -228879.45744501668,
               15660.672213170634,
               1005048.1826439175,
               457135.4227123128,
               175720.62255952347,
               -973487.5604810078,
               389259.79627994023],
              'SUMSQ': [188041468.90074167,
               222140297.68636304,
               345146739.91127175,
               256364868.87938944,
               297644766.21401066,
               313497158.8976518,
               171394108.25150213,
               217147439.52766788,
               268182500.5338874,
               158143857.26592916],
              'centroid': (-9.341826324656967,
               -12.184580251218154,
               -24.748817912262744,
               -26.475356558128013,
               1.8115294636403279,
               116.25774235325824,
           

In [ ]:
class DiscardSet():
    def __init__(self, stat, points_clusters):
        self.points_clusters = points_clusters
        self.n_points = len(points_clusters)
        self.n_dim = len(stat[0]['centroid'])
        self.N = {c: s['N'] for c,s in stat.items()}
        self.SUM = {c: s['SUM'] for c,s in stat.items()}
        self.SUMSQ = {c: s['SUMSQ'] for c,s in stat.items()}
        self.centroids = {c: s['centroid'] for c,s in stat.items()}
        self.std = {c: [math.sqrt((s['SUMSQ'][i]/s['N']) - s['centroid'][i]**2) for i in range(self.n_dim)] \
                    for c,s in stat.items()}
#         self.stale = False
    
    def update_centroids(self):
        for c,ps in self.clusters_points.items():
            self.N[c] = len(ps)
            self.SUM[c] = [sum(i) for i in list(map(list, zip(*ps)))]
            self.SUMSQ[c] = [sum([v**2 for v in i]) for i in list(map(list, zip(*ps)))]
            self.centroids[c] = tuple([self.SUM[c][i] / self.N[c] for i in range(self.n_dim)])
            self.std[c] = [math.sqrt((self.SUMSQ[c][i]/self.N[c]) - self.centroid[i]**2) for i in range(self.n_dim)]
                    
        
    def update_clusters_points(self, new_points_clusters):
        if points_clusters:
            combined_result = defaultdict(list)
            for k, v in itertools.chain(self.points_clusters.items(),
                                        points_clusters.items()):
                combined_result[k] += v
            self.points_clusters = combined_result

In [ ]:
discard_set = DiscardSet(cluster_stat, points_clusters)
# compression_set = CS()
# retained_set = RS()
# intermediate_records = IntermediateRecords()

######################         displaySet(self.DS_set)         ######################

In [ ]:
def make_CS(self):
    if len(self.RS_set) <= 3 * self.K:
        return
    # print("=> We need to make more CS at here")
    X = self.RS_set
    keamns = _KMeans(K= 3 * self.K, tol = 1, n_init=1, init="kmeans++")
    Y = keamns.fit(X)
    dict_res = make_predict_dict(K * 3, self.RS_set, Y)
    self.RS_set = []
    for l in dict_res:
        if len(dict_res[l]) == 1:
            self.RS_set.append(dict_res[l][0])
        elif len(dict_res[l]) > 1:
            self.CS_set.append(DiscardSet("CS", dict_res[l]))

In [ ]:
    def initialisation(self):
        keamns = _KMeans(K=self.K, tol = 1, compulsory=True, n_init=15)
        Y = keamns.fit(X)
        self.DS_set = initialise_DS(X, Y, self.K) # {}

        
        self.make_CS()
        self.information_summary()
        # print("**** End of First Round  ****")
        
        
        
        
    
    
    def process_one_round(self, train_data, next_line, ROUND_BATCH):
        X = train_data[next_line : next_line + ROUND_BATCH]
        ds_batch_process_add = {}
        cs_batch_process_add = {}

        for point in X:
            nearest_DS_label = find_nearest_DS(self.DS_set, self.alpha, point)
            if nearest_DS_label >= 0:  # valid: If they can enter DS ==> enter DS
                if nearest_DS_label not in ds_batch_process_add:
                    ds_batch_process_add[nearest_DS_label] = []
                ds_batch_process_add[nearest_DS_label].append(point)
            else:
                # If they can enter CS ==> enter CS
                nearest_CS_index = find_nearest_CS(self.CS_set, self.alpha, point)
                if nearest_CS_index >= 0:
                    if nearest_CS_index not in cs_batch_process_add:
                        cs_batch_process_add[nearest_CS_index] = []
                    cs_batch_process_add[nearest_CS_index].append(point)
                else:
                    # Else: enter RS
                    self.RS_set.append(point)
        for _label in ds_batch_process_add:
            self.DS_set[_label].merge_points(ds_batch_process_add[_label])
        for _index in cs_batch_process_add:
            self.CS_set[_index].merge_points(cs_batch_process_add[_index])


    def rounds(self):
        if self.current_file_index >= len(self.files):
            return
        # print("===> Now Operate on File: " + str(self.files[self.current_file_index]))
        self.round_no += 1
        train_data = load_data(self.files[self.current_file_index])
        self.current_file_index += 1
        ROUND_BATCH = int(len(train_data) * 0.1)
        DATA_LENGTH = len(train_data)
        next_line = 0
        while next_line < DATA_LENGTH:
            self.process_one_round(train_data, next_line, ROUND_BATCH)
            next_line += ROUND_BATCH

        self.merge_CS()
        self.make_CS()
        # self.merge_CS() # Change to Merge First Than Make
        self.information_summary()
        displaySet(self.DS_set)
        self.rounds()  # Trigger next round

    def teardown(self):
        ds_batch_process_add = {}
        for cs_index in range(len(self.CS_set)):
            self.CS_set[cs_index].update_statistics()
            centroid_point = self.CS_set[cs_index].centroid
            nearest_label = find_nearest_DS(self.DS_set, float("inf"), centroid_point)
            if nearest_label >= 0:
                if nearest_label not in ds_batch_process_add:
                    ds_batch_process_add[nearest_label] = []
                ds_batch_process_add[nearest_label].append(cs_index)

        # self.information_summary()
        for ds_label in ds_batch_process_add:
            for cs_index in ds_batch_process_add[ds_label]:
                self.DS_set[ds_label].merge_Cluster(self.CS_set[cs_index])

        output_cluster_result(self.DS_set, self.RS_set, self.cluster_result_filepath)
        output_intermediate_result(
            self.intermediate_info, self.intermediate_filepath
        )  # This is the last step

    def run(self):
        self.initialisation()
        self.rounds()
        self.teardown()

    

In [ ]:
discard_set = DS()
compression_set = CS()
retained_set = RS()
intermediate_records = IntermediateRecords()

In [ ]:
if index == 0:
    n_data = data_points.count()
    n_sample = 10000 if n_data > 10000 else int(n_data *.1)
    data_sample = data_points.filter(lambda kv: kv[0]<n_sample).collectAsMap()
    points = [list(p) for i,p in data_sample.items()]

    cluster_stat, points_clusters = KMeans(k=n_cluster, max_iterations=5).fit(points)

    # c. Use the K-Means result from b to generate the DS clusters
    # (i.e., discard points and generate statistics)
    discard_set.init(ds_center, ds_stat, ds_cluster)

    # d. The initialization of DS has finished, so far, you have K clusters in DS.
    # e. Run K-Means on the rest of the data points with a large number of clusters
    # (e.g., 5 times of K) to generate CS (clusters with more than one points)
    rest_data = row_data_rdd.filter(lambda kv: kv[0] >= first_N).collectAsMap()
    center, stat, cluster = KMeans(k=num_of_cluster * 3, max_iterations=3).fit(rest_data)

    # and generate RS (clusters with only one point).
    cs_center, cs_stat, cs_cluster, remaining2 = checkBelongings(center, stat, cluster)
    compression_set.init(cs_center, cs_stat, cs_cluster)
    retained_set.add(remaining2)

In [ ]:
    alpha = 3
    discard_set = DS()
    compression_set = CS()
    retained_set = RS()
    intermediate_records = IntermediateRecords()


In [ ]:
with open(output_file, 'w+') as f:
    for u_b_s in pred:
        f.write(json.dumps({'user_id': u_b_s[0], 'business_id': u_b_s[1], 'stars': u_b_s[2]}) + '\n')

In [ ]:
        ###'k-means++':
#         centers, _ = _kmeans_plusplus(X, n_clusters,
#                                       random_state=random_state,
#                                       x_squared_norms=x_squared_norms)
        centers = [[0]*n_features] *n_clusters
        indices = [-1]*n_clusters

        # Set the number of local seeding trials if none is given
        n_local_trials = 2 + int(np.log(n_clusters))

        # Pick first center randomly and track index of point
        center_id = random.randint(0, n_samples-1)
        centers[0] = X[center_id]
        indices[0] = center_id

        # Initialize list of closest distances and calculate current potential
        closest_dist_sq = euclidean_distances(centers[0, np.newaxis], X)
        current_pot = closest_dist_sq.sum()

        # Pick the remaining n_clusters-1 points
        for c in range(1, n_clusters):
            # Choose center candidates by sampling with probability proportional
            # to the squared distance to the closest existing center
            rand_vals = random_state.random_sample(n_local_trials) * current_pot
            candidate_ids = np.searchsorted(stable_cumsum(closest_dist_sq),
                                            rand_vals)
            # XXX: numerical imprecision can result in a candidate_id out of range
            np.clip(candidate_ids, None, closest_dist_sq.size - 1,
                    out=candidate_ids)

            # Compute distances to center candidates
            distance_to_candidates = euclidean_distances(
                X[candidate_ids], X, Y_norm_squared=x_squared_norms, squared=True)

            # update closest distances squared and potential for each candidate
            np.minimum(closest_dist_sq, distance_to_candidates,
                       out=distance_to_candidates)
            candidates_pot = distance_to_candidates.sum(axis=1)

            # Decide which candidate is the best
            best_candidate = np.argmin(candidates_pot)
            current_pot = candidates_pot[best_candidate]
            closest_dist_sq = distance_to_candidates[best_candidate]
            best_candidate = candidate_ids[best_candidate]

            # Permanently add best center candidate found in local tries
            centers[c] = X[best_candidate]
            indices[c] = best_candidate

        return centers, indices